# Scraping the required content for the text mining project

Trying to scrape the Grimm's fairytales from [this site](http://www.worldoftales.com/fairy_tales/Grimm_fairy_tales_Margaret_Hunt.html) using [this tutorial](http://web.stanford.edu/~zlotnick/TextAsData/Web_Scraping_with_Beautiful_Soup.html) and [the bs4 documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

In [21]:
# Playing around with how to extract the text from a specific fairytale link.
from bs4 import BeautifulSoup
import urllib

r = urllib.urlopen('http://www.worldoftales.com/fairy_tales/Brothers_Grimm/Margaret_Hunt/The_Spindle,_The_Shuttle,_and_the_Needle.html').read()
soup = BeautifulSoup(r)
print type(soup)

<class 'bs4.BeautifulSoup'>


In [22]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <!-- InstanceBegin template="/Templates/world of tales.dwt" codeOutsideHTMLIsLocked="false" -->
 <head>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <!-- InstanceBeginEditable name="meta" -->
  <meta content="Fairy tales &gt; Grimm fairy tales &gt; The Spindle, The Shuttle, and the Needle at  World of Tales - Stories for children from around the world!" name="Description"/>
  <meta content="fairy tales, brothers grimm, grimm fairy tales, stories for children" name="Keywords"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Brothers Grimm fairy tales - The Spindle, The Shuttle, and the Needle
  </title>
  <!-- InstanceEndEditable -->
  <!--[if IE]> 
<style type="text/css">.big { position:absolute; left:10px; right:0px;}.box8 {padding-left:160px;} .show_opt {

In [36]:
tale = soup.find_all("div", id = 'text', class_ = "GM")

In [37]:
tale

[<div class="GM" id="text">\n         <p align="justify">There was once a girl whose father and mother died while she was still a little child.  All alone,  in a small house at the end of the village, dwelt her godmother, who supported herself by  spinning, weaving, and sewing.  The old woman took the forlorn child to live with her, kept her  to her work, and educated her in all that is good.  When the girl was fifteen years old, the old  woman became ill, called the child to her bedside, and said, "Dear daughter, I feel my end  drawing near.  I leave thee the little house, which will protect thee from wind and weather, and  my spindle, shuttle, and needle, with which thou canst earn thy bread."  Then she laid her hands  on the girl's head, blessed her, and said, "Only preserve the love of God in thy heart, and all will  go well with thee."  Thereupon she closed her eyes, and when she was laid in the earth, the  maiden followed the coffin, weeping bitterly, and paid her the last mark o

In [38]:
# Now having a look at how to extract all of the URLs and put them in a list.
from bs4 import BeautifulSoup
import urllib

r = urllib.urlopen('http://www.worldoftales.com/fairy_tales/Grimm_fairy_tales_Margaret_Hunt.html').read()
soup = BeautifulSoup(r)

In [54]:
urls = []
url = "http://www.worldoftales.com/fairy_tales/"
for tag in soup.findAll('a', href = re.compile('Brothers_Grimm')): # Specifically only want URLS for the Grimm's fairytales
    match = urlparse.urljoin(url, tag['href'])
    urls.append(match)

In [56]:
urls

[u'http://www.worldoftales.com/fairy_tales/Brothers_Grimm/Margaret_Hunt/The_Frog-King,_or_Iron_Henry.html',
 u'http://www.worldoftales.com/fairy_tales/Brothers_Grimm/Margaret_Hunt/Cat_and_Mouse_in_Partnership.html',
 u"http://www.worldoftales.com/fairy_tales/Brothers_Grimm/Margaret_Hunt/Our_Lady's_Child.html",
 u'http://www.worldoftales.com/fairy_tales/Brothers_Grimm/Margaret_Hunt/The_Story_of_the_Youth_who_Went_Forth_to_Learn_What_Fear_Was.html',
 u'http://www.worldoftales.com/fairy_tales/Brothers_Grimm/Margaret_Hunt/The_Wolf_and_The_Seven_Little_Kids.html',
 u'http://www.worldoftales.com/fairy_tales/Brothers_Grimm/Margaret_Hunt/Faithful_John.html',
 u'http://www.worldoftales.com/fairy_tales/Brothers_Grimm/Margaret_Hunt/The_Good_Bargain.html',
 u'http://www.worldoftales.com/fairy_tales/Brothers_Grimm/Margaret_Hunt/The_Wonderful_Musician.html',
 u'http://www.worldoftales.com/fairy_tales/Brothers_Grimm/Margaret_Hunt/The_Twelve_Brothers.html',
 u'http://www.worldoftales.com/fairy_tales/B

In [59]:
# Now extract all of the texts from each URL and put in a list.
tales = []
for url in urls:
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r)
    tale = soup.find_all("div", id = 'text', class_ = "GM")
    tales.append(tale)

In [63]:
tales[0]

[<div class="GM" id="text">\n         <p align="justify">In old times when wishing still helped one, there lived a king whose  daughters were all beautiful, but the youngest was so beautiful that the  sun itself, which has seen so much, was astonished whenever it shone in  her face.  Close by the King's castle lay a great dark forest, and under an   old lime-tree in the forest was a well, and when the day was very warm,  the King's child went out into the forest and sat down by the side of the  cool fountain, and when she was dull she took a golden ball, and threw it  up on high and caught it, and this ball was her favourite plaything.</p>\n         <p> Now it so happened that on one occasion the princess's golden ball did   not fall into the little hand which she was holding up for it, but on to the  ground beyond, and rolled straight into the water.  The King's daughter  followed it with her eyes, but it vanished, and the well was deep, so deep  that the bottom could not be seen.  On

## To do
* Extract titles for each tale (in a separate list for ease of feeding into Pandas dataframe)
* Work out if it is possible to pull out cleaner text from the URLs without all of the HTML tags